<a href="https://www.kaggle.com/code/mdsayeduzzaman/notebookddacd2cfa9?scriptVersionId=104498859" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("complete")

/kaggle/input/cvpdfdata/agaspar_cv.pdf
complete


In [2]:
#all pip install
!pip install pdfplumber
print("complete")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.5 MB/s eta 0:00:00
complete


In [3]:
#all import
import pdfplumber
import pandas as pd
import os
import spacy
from spacy.matcher import Matcher

from datetime import datetime

import re
from nltk.corpus import stopwords



print("complete")

complete


In [4]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
print("complete")

complete


In [5]:
def extract_pdf(pdf_path):
    linesOfFile = []
    with pdfplumber.open(pdf_path) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            for line in single_page_text.split('\n'):
                linesOfFile.append(line)
                #print(linesOfFile)
    return linesOfFile


#folder_with_pdfs = '../input/cvalltogether'
folder_with_pdfs = '../input/cvpdfdata'


linesOfFiles = []



listOfPdfFiles=[]

for pdf_file in os.listdir(folder_with_pdfs):
    if pdf_file.endswith('.pdf'):
        #print(pdf_file)
        listOfPdfFiles.append(pdf_file)
        pdf_file_path = os.path.join(folder_with_pdfs, pdf_file)
        linesOfFile = extract_pdf(pdf_file_path)
        linesOfFiles.append(linesOfFile)
        
        
        
        
df = pd.DataFrame(linesOfFiles)
#df.to_csv('test.csv')
print("complete")

complete


In [6]:
listOfPdfFiles

['agaspar_cv.pdf']

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,264,265,266,267,268,269,270,271,272,273
0,András Gáspár 12/10/2016,Contact StewardObservatory Tel: (520)-360-0983,Information TheUniversityofArizona agaspar@as....,"933NCherryAvenue,Rm278 http://merope.as.arizon...","Tucson,AZ85721-0065 U.S.permanentresident","Research Circumstellar Disks, Exozodi, Extraso...","Areas ters,ISM,Coronagraphy,InfraredDetectors,...","Education 2011 Ph.D.,Astronomy,StewardObservat...",Thesis: “Observationsandmodelsofinfrareddebris...,theirevolution”,...,"∗10. AVariableStarSurveyoftheOpenClusterNGC2126,","Gáspár, A., Kiss, L. L., Derekas, A., Bedding,...","G.M.,andVáradi,M.","2004,CommunicationsinAstroseismology,145,70","11. AVariableStarSurveyoftheOpenClusterNGC2126,","Derekas,A.,Kiss,L.L.,Bedding,T.R.,Gáspár,A.,Sá...","Kaspi,S.,andKiss,Cs.","2004, Variable Stars in the Local Group, IAU C...","Proceedings,Vol.310,p.162",8


In [8]:
# to do : do partial match , i.e. word+etc

reference=[ 
            #node1
            "articles","books","chapters","citations","editorials","journals",
               "seminars","scholarly",
            #node2
            "awards","proposals","grants","honors","scholarships","sponsored",
            #node3
            "appointments","experiences","services","employments","practices",
                "professionals",
            #node4
            "affiliations","memberships","committees",
            #node5
            "contacts","introductions",
            #node6
            "publications","conferences","presentations","newsletters","reports",
            #node7
            "educations","certificates","certifications",
            #node8
            "researches",
            #node9
            "teaching","outreaches"
          ]
#to store data
allCVData=[]
allCVData.append(("fileName","personName","sectionName","head","relation/lebel","tail"))

print(allCVData)
print(reference,len(reference))

print("complete")

[('fileName', 'personName', 'sectionName', 'head', 'relation/lebel', 'tail')]
['articles', 'books', 'chapters', 'citations', 'editorials', 'journals', 'seminars', 'scholarly', 'awards', 'proposals', 'grants', 'honors', 'scholarships', 'sponsored', 'appointments', 'experiences', 'services', 'employments', 'practices', 'professionals', 'affiliations', 'memberships', 'committees', 'contacts', 'introductions', 'publications', 'conferences', 'presentations', 'newsletters', 'reports', 'educations', 'certificates', 'certifications', 'researches', 'teaching', 'outreaches'] 36
complete


In [9]:
def dfPartByPart(textList):
    collectedText=[]
    collectedByDictionary=dict()
    collectedParts=dict()
    keyDictionary="introductions"
    temp=[]



    for smallPartText in textList:
        if smallPartText==None:
            continue
   
        for partOfSmallPartText in smallPartText.split():
            if len(partOfSmallPartText)<2:
                continue
            result = list(filter(lambda x: x.startswith(partOfSmallPartText.lower()), reference))
            if result:
                
                collectedText.append(temp)
                if keyDictionary in collectedByDictionary:
                    collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
                else:
                    collectedByDictionary[keyDictionary]=temp
                keyDictionary=result[0]
                temp=[]
                break
        
        temp.append(smallPartText)
    
    
    collectedText.append(temp)
    if keyDictionary in collectedByDictionary:
        collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
    else:
        collectedByDictionary[keyDictionary]=temp

    return collectedByDictionary
print("complete")

complete


In [10]:
def reDistribute(prevSectionDictionary):
    revisedSectionDictionary=dict()

    for sectionPart in prevSectionDictionary:
        if sectionPart in ["articles","books","chapters","citations","editorials","journals","seminars","scholarly"]:
        
            temp="articles / books / chapters / citations / editorials / journals / seminars / scholarly"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["awards","proposals","grants","honors","scholarships","sponsored"]:
        
            temp="awards / grants / proposals / honors / scholarships / sponsored"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
    
        elif sectionPart in ["appointments","experiences","services","employments","practices","professionals"]:
        
            temp="appointments / experiences / services / employments / practices / professionals"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["affiliations","memberships","committees"]:
        
            temp="affiliations / memberships / committees"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["contacts","introductions"]:
        
            temp="contacts / introductions"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["publications","conferences","presentations","newsletters","reports"]:
        
            temp="publications / conferences / presentations / newsletters / reports"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["educations","certificates","certifications"]:
        
            temp="educations / certificates / certifications"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["researches"]:
        
            temp="researches"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
        elif sectionPart in ["teaching","outreaches"]:
        
            temp="teaching / outreaches"
            if temp in revisedSectionDictionary:
            
                revisedSectionDictionary[temp].extend(prevSectionDictionary[sectionPart])
            else:
                revisedSectionDictionary[temp]=prevSectionDictionary[sectionPart]
    return revisedSectionDictionary
    
    
print("complete")

complete


In [11]:
matrix_res = df.to_numpy()

print(len(matrix_res))

for i in range(len(matrix_res)):
   
    
    sectionDictionary=dfPartByPart(matrix_res[i])
    
#print(sectionDictionary)
print("complete")

1
complete


In [12]:
print(sectionDictionary["contacts"])
print(sectionDictionary["introductions"])


['Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']
['András Gáspár 12/10/2016', '2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference', 'Proceedings,Vol.310,p.162', '8']


In [13]:
newSectionDictionary=reDistribute(sectionDictionary)

In [14]:
print(sectionDictionary["contacts"])
print(sectionDictionary["introductions"])
print("****************************")


print(newSectionDictionary["contacts / introductions"])
print("complete")

['Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']
['András Gáspár 12/10/2016', '2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference', 'Proceedings,Vol.310,p.162', '8', 'Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']
****************************
['András Gáspár 12/10/2016', '2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference', 'Proceedings,Vol.310,p.162', '8', 'Contact StewardObservatory Tel: (520)-360-0983', 'Information TheUniversityofArizona agaspar@as.arizona.edu', '933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar', 'Tucson,AZ85721-0065 U.S.permanentresident']


In [15]:
print(newSectionDictionary.keys())

dict_keys(['contacts / introductions', 'researches', 'educations / certificates / certifications', 'appointments / experiences / services / employments / practices / professionals', 'awards / grants / proposals / honors / scholarships / sponsored', 'teaching / outreaches', 'publications / conferences / presentations / newsletters / reports'])


In [16]:
#import re
#from nltk.corpus import stopwords
# load pre-trained model
#nlp = spacy.load('en_core_web_sm')
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))
# Education Degrees
EDUCATIONDEGREE = [
            'BE','B.E.', 'B.E', 'BS', 'B.S','B.S.','BSC','B.SC','B.SC.','C.A.','B.COM','BCOM',
            'M.COM', 'MCOM','M.COM.',
            'ME', 'M.E', 'M.E.', 'MS', 'M.S','M.S.','MSC','M.SC','M.SC.',
            'BTECH', 'B.TECH','B.TECH.', 'M.TECH','M.TECH.', 'MTECH',
            'PHD','PH.D', 'PH.D.','MBA','GRADUATE', 
            'POST-GRADUATE','MASTERS',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]
def extract_educationDegree(resume_text):
    nlp_text = nlp(resume_text)
    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    edu = []
    
    # Extract education degree
    for index, text in enumerate(nlp_text):
        text=text.replace(","," ")
        #print(text)
        for tex in text.split():
            # Replace all special symbols
            #tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            
            if tex.upper() in EDUCATIONDEGREE and tex not in STOPWORDS:
                
                if tex not in edu:
                    edu.append(tex)
                                
                
    return edu
print("complete")

complete


In [17]:
#import re
#from datetime import datetime
def getDate(dataMaybeDate):
    

    match = re.match(r'.*([1-3][0-9]{3})', dataMaybeDate)
    if match is not None:
        # Then it found a match!
        #print(match.group(1))
        print(match)
        return match.group()
    return None

print("complete")

complete


In [18]:
def newGetDate(dataMaybeDate):
    match = re.search('\d{4}',dataMaybeDate)
    if match is not None:
        # Then it found a match!
        #print(match.group(1))
        #print(match)
        return match.group()
    return None

In [19]:
def findInstitute(tika_text):
    sub_patterns = ['[A-Z][a-z]* University',
                    '[A-Z][a-z]* Educational Institute',
                '[A-Z][a-z]* College',
                'University of [A-Z][a-z]*',
                'The University of [A-Z][a-z]*',
                    'TheUniversityof[A-Z][a-z]*',
                'Ecole [A-Z][a-z]*',
                   '[A-Z][a-z]*University',
                    '[A-Z][a-z]*EducationalInstitute',
                '[A-Z][a-z]*College',
                'Universityof[A-Z][a-z]*',
                'Ecole[A-Z][a-z]*']
    pattern = '({})'.format('|'.join(sub_patterns))
    matches = re.findall(pattern, tika_text)

    return matches

print("complete")

complete


In [20]:
def removePrefix(dataSent):
    resDataIndex=0
    for i in dataSent:
        if i.isalpha():
            #print(i,resDataIndex)
            break
        resDataIndex+=1
    dataSent=dataSent[resDataIndex:]
    return dataSent
print("complete")

complete


In [21]:
#Name
#import spacy
#from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)
    res=[]
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        temp=span.text
        if ("vitae" in temp.lower() or "sciences" in temp.lower() 
            or "engineering" in temp.lower() or "biographical" in temp.lower() ):
            continue
        res.append(temp)
        #res.append(span.text)
    return res
    
    

print("complete")

complete


In [22]:
def handleEducations(listOfEducations):
    eduData=[]
    pieceEduData=dict()
    rest=""
    for eduPart in listOfEducations:
        #print(type(eduPart))
        
        res=extract_educationDegree(eduPart)
        if res:
            if len(pieceEduData)!=0:
                if len(rest)>1:
                    pieceEduData["total details"]=rest
                    rest=""
                eduData.append(pieceEduData)
                pieceEduData=dict()
            pieceEduData["degree"]=res[0]
            index001=eduPart.find(res[0])
            eduPart=eduPart[:index001]+eduPart[index001+len(res[0]):]
            #print(res,index001,eduPart)
            #print(res,eduPart)
        #datePart=getDate(eduPart)
        newDatePart=newGetDate(eduPart)
        
        #if datePart:
        #    print("date:",datePart)
        if newDatePart:
            pieceEduData["date"]=newDatePart
            index002=eduPart.find(newDatePart)
            eduPart=eduPart[:index002]+eduPart[index002+len(newDatePart):]
            #print(index002,"date:",newDatePart,eduPart)
        
        #test001=eduPart.split(":")
        #for itest in test001:
        if "advisor" in eduPart.lower():
            
            index003=eduPart.lower().find("advisor")
            advis001=eduPart[index003+len("advisor"):]
            advis001=removePrefix(advis001)
            pieceEduData["advisor"]=advis001
            eduPart=eduPart[:index003]
            #print("advisor",advis001,eduPart)
        org=findInstitute(eduPart)
        if org:
            pieceEduData["organization"]=org[0]
            index004=eduPart.find(org[0])
            eduPart=eduPart[:index004]+eduPart[index004+len(org[0]):]
            #print(org,eduPart)
        #print(eduPart)
        eduPart=removePrefix(eduPart)
        if len(eduPart)>1:
            rest+=eduPart
        #print(eduPart)
    if len(pieceEduData)!=0:
        if len(rest)>1:
            pieceEduData["total details"]=rest
                    #rest=""
        eduData.append(pieceEduData)
    return eduData
print("complete")

complete


In [23]:
ret001=handleEducations(newSectionDictionary['educations / certificates / certifications'])
print(ret001)
print("complete")

[{'degree': 'Ph.D.', 'date': '2011', 'organization': 'TheUniversityofArizona', 'advisor': 'Prof.GeorgeH.Rieke', 'total details': 'Education  ,Astronomy,StewardObservatory,Thesis: “Observationsandmodelsofinfrareddebrisdisksignaturesandtheirevolution”'}, {'degree': 'M.Sc.', 'date': '2008', 'organization': 'TheUniversityofArizona', 'advisor': 'Prof.GeorgeH.Rieke', 'total details': 'Astronomy,StewardObservatory,'}, {'degree': 'B.Sc.', 'date': '2006', 'organization': 'UniversityofSzeged', 'advisor': 'Dr.ZoltánBalog', 'total details': 'Astronomy,,Hungary'}]
complete


In [24]:
#textTest="Education 2011 Ph.D.,Astronomy,StewardObservatory,TheUniversityofArizona"
#textTest="2008 M.Sc.,Astronomy,StewardObservatory,TheUniversityofArizona"
textTest="2006 B.Sc.,Astronomy,UniversityofSzeged,Hungary"

ret=extract_educationDegree(textTest)
print(ret)

handleEducations([textTest])

['B.Sc.']


[{'degree': 'B.Sc.',
  'date': '2006',
  'organization': 'UniversityofSzeged',
  'total details': 'Astronomy,,Hungary'}]

In [25]:

test_str = "abc 2006 B.Sc.,Astronomy,UniversityofSzeged,Hungary, 2011"
    
match = re.search('\d{4}',test_str)
print(match)
print("complete")

<re.Match object; span=(4, 8), match='2006'>
complete


In [26]:
def handleContacts(sentDataContacts):
    #print(sentDataContacts)
    textContacts001="".join(sentDataContacts)
    nameContacts001=extract_name(textContacts001)
    print(nameContacts001)
    for contacts00 in sentDataContacts:
        print(contacts00)
print("complete")

complete


In [27]:
contacts001=handleContacts(newSectionDictionary['contacts / introductions'])
print(contacts001)
print("complete")

['András Gáspár', 'Variable Stars', 'Local Group', 'IAU Colloquium', 'New Zealand', 'ASP ConferenceProceedings', 'StewardObservatory Tel']
András Gáspár 12/10/2016
2004, Variable Stars in the Local Group, IAU Colloquium 193, New Zealand, ASP Conference
Proceedings,Vol.310,p.162
8
Contact StewardObservatory Tel: (520)-360-0983
Information TheUniversityofArizona agaspar@as.arizona.edu
933NCherryAvenue,Rm278 http://merope.as.arizona.edu/˜agaspar
Tucson,AZ85721-0065 U.S.permanentresident
None
complete


In [28]:
print("complete")

complete


In [29]:
def handlePublications(sentDataOfPublications):
    for dataPart in sentDataOfPublications:
        print(dataPart)
        print("************************************")
print("complete")

complete


In [30]:
ret002=handlePublications(newSectionDictionary['publications / conferences / presentations / newsletters / reports'])
print(ret002)
print("complete")

Publications
************************************
Citationsofrefereedarticles(FirstAuthored/total): 193/667
************************************
h-indexofrefereedarticles(FirstAuthored/total): 7/13
************************************
Refereed First Authored Papers
************************************
∗1. TheCorrelationBetweenMetallicityandDebrisDiskmass,
************************************
Gáspár,András,Rieke,George,andBallering,Nicholas
************************************
2016,TheAstrophysicalJournal,826,171G
************************************
∗2. TheHerschelColdDebrisDisks: ConfusionwiththeExtragalacticBackgroundat160µm,
************************************
Gáspár,András,andRieke,George
************************************
2014,TheAstrophysicalJournal,784,33G
************************************
∗3. TheCollisionalEvolutionofDebrisDisks,
************************************
Gáspár,András,Rieke,George,andBalog,Zoltan
************************************
2013,TheAstrophysicalJourna

In [31]:
#import en_core_web_sm
#nlp = en_core_web_sm.load()
text001='555-1239Moe Szyslak(636) 555-0113Burns, C. Montgomery555 -6542Rev. Timothy Lovejoy555 8904Ned Flanders636-555-3226Simpson, Homer5553642Dr. Julius Hibbert'
doc = nlp(text001)
res003=[(X.text, X.label_) for X in doc.ents if X.label_ == 'PERSON']
print(res003)
print("********************************")
res004=extract_name(text001)

print(res004)

print("complete")

[('8904Ned', 'PERSON'), ('Julius Hibbert', 'PERSON')]
********************************
['C. Montgomery555', 'Timothy Lovejoy555', 'Lovejoy555 8904Ned', '8904Ned Flanders636', 'Julius Hibbert']
complete


In [32]:
text005='Derekas,A.,Kiss,L.L.,Bedding,T.R.,Gáspár,A.,Sárneczky,K.,Szabó,Gy.M.,Váradi,M.,Kaspi,S.,andKiss,Cs.'


print("********************************")
res005=extract_name(text005)
print(res005)


doc005 = nlp(text005)
res006=[(X.text, X.label_) for X in doc005.ents if X.label_ == 'PERSON']
print("********************************")
print(res006)

text006='1) *Gouge, D. H. and N. G. M. Hague. 1992. Biological control of sciarids in ornamentalsusing the entomopathogenic nematode Steinernema feltiae. Bulletin International Organization for Biological and Integrated Control of Noxious Animals and Plants / Section Régionale Ouest Paléarctique. 16: 86-91'

print("********************************")
res007=extract_name(text006)
print(res007)


doc006 = nlp(text006)
res008=[(X.text, X.label_) for X in doc006.ents if X.label_ == 'PERSON']
print("********************************")
print(res008)


print("complete")

********************************
[]
********************************
[]
********************************
['D. H.', 'N. G.', 'G. M.', 'M. Hague', 'Bulletin International', 'International Organization', 'Integrated Control', 'Noxious Animals', 'Section Régionale', 'Régionale Ouest', 'Ouest Paléarctique']
********************************
[('D. H.', 'PERSON'), ('N. G. M. Hague', 'PERSON'), ('Steinernema', 'PERSON')]
complete


In [33]:
print("complete")

complete
